### import modules

In [57]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
import numpy as np
import pmdarima as pm

### ignore warnings

In [58]:
import warnings
warnings.filterwarnings('ignore')

### config

In [76]:
LABEL_DATA_PATH = 'D:/projects/rahnamcollege-ml/demand-prediction/data/label/label.parquet'
ARIMA_MODELS_PATH = 'D:/projects/rahnamcollege-ml/demand-prediction/model/arima_models/'
OUTPUT_PATH = 'D:/projects/rahnamcollege-ml/demand-prediction/data/arima_output.xlsx'
TRAIN_TEST_SPLIT_DATE = '2023-04-01'
DATA_LAST_DATE = '2023-04-31'
ADDED_PASSENGER_COUNT = 100
HIGH_DEMAND_THRESHOLD = 1000
LOW_DEMAND_THRESHOLD = 300

TESTING_ARIMA_ORDERS = [
    (4, 2, 1),
    (5, 1, 2),
    (4, 2, 1),
    (1, 2, 2),
    (6, 1, 4)
]

### load data

In [60]:
rides_df = pd.read_parquet(LABEL_DATA_PATH)
rides_df = rides_df[rides_df['date'] < DATA_LAST_DATE].reset_index(drop=True)
print(rides_df.shape)
rides_df.head()

(31178, 3)


,date,PULocationID,count
0,2023-01-02,70,503.0
1,2023-01-02,132,6419.0
2,2023-01-02,142,2028.0
3,2023-01-02,164,1462.0
4,2023-01-02,144,567.0


### prepare data

In [61]:
def train_test_split(df, split_date):
    train_df = df[df['date'] < split_date]
    test_df = df[df['date'] >= split_date]
    return train_df, test_df

train_df, test_df = train_test_split(rides_df, TRAIN_TEST_SPLIT_DATE)

print(train_df.shape)
train_df.head()

(23318, 3)


,date,PULocationID,count
0,2023-01-02,70,503.0
1,2023-01-02,132,6419.0
2,2023-01-02,142,2028.0
3,2023-01-02,164,1462.0
4,2023-01-02,144,567.0


### seprate locations data

In [62]:
train_location_dfs = {}
test_location_dfs = {}
location_dfs = {}


location_ids = train_df['PULocationID'].unique()

for location_id in location_ids:
    location_dfs[location_id] = rides_df[rides_df['PULocationID'] == location_id].sort_values(by=['date']).reset_index(drop=True)
    train_location_dfs[location_id] = train_df[
        train_df['PULocationID'] == location_id].sort_values(by=['date']).reset_index(drop=True)
    
    test_location_dfs[location_id] = test_df[
        test_df['PULocationID'] == location_id].sort_values(by=['date']).reset_index(drop=True)

### train model

In [79]:
def train_arima_model(train_df, location_rides_df):  
    auto_model = pm.auto_arima(train_df['count'], seasonal=False, max_p=7)
    
    fitted_values = auto_model.predict_in_sample()
    order = auto_model.order
    
    y_pred = []
    
    for idx in range(len(train_df), len(location_rides_df)):
        train_data = location_rides_df.iloc[:idx]
        model_arima = ARIMA(train_data['count'], order=order)
        fitted_model = model_arima.fit()
        y_pred.append(fitted_model.forecast(step=1).values[0])
    
    return fitted_values.values, y_pred
         

for loc_id, location_df in location_dfs.items():
    print("loc_id", loc_id)
    test_df = test_location_dfs[loc_id]
    train_df = train_location_dfs[loc_id]
    fitted_values, prediction = train_arima_model(train_df, location_df)
    test_df['pred'] = prediction
    train_df['pred'] = fitted_values
    

### concat test dataframes

In [66]:
pred_df = pd.concat(test_location_dfs.values())
print(pred_df.shape)
pred_df.head()

(7860, 4)


,date,PULocationID,count,pred
0,2023-04-01,70,373.0,538.110268
1,2023-04-02,70,528.0,507.960339
2,2023-04-03,70,568.0,565.995568
3,2023-04-04,70,467.0,507.184754
4,2023-04-05,70,421.0,487.668488


### evaluate loss

In [67]:
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error

In [68]:
def evaluate_model(actual, pred):
    mae = mean_absolute_error(actual, pred)
    mape = mean_absolute_percentage_error(actual, pred)
    mse = mean_squared_error(actual, pred)

    print(f'mape = {mape}')
    print(f'mae = {mae}')
    print(f'rmse = {np.sqrt(mse)}')

evaluate_model(pred_df['count'], pred_df['pred'])

mape = 511864248394185.3
mae = 45.493224120023044
rmse = 139.9596039882008


### create result dataframe

In [69]:
dfs = []

for loc_id in location_ids:
    train_df = train_location_dfs[loc_id]
    test_df = test_location_dfs[loc_id]
    dfs.append(train_df)
    dfs.append(test_df)
    

result_df = pd.concat(dfs)
result_df['pred'] = result_df['pred'].apply(lambda x: max(0, int(x)))

### evaluate on high demand locations

In [70]:
mean_counts = rides_df.groupby('PULocationID')['count'].mean()
true_items_location_ids = mean_counts[mean_counts > HIGH_DEMAND_THRESHOLD].index.tolist()
high_demand_df = result_df[result_df['PULocationID'].isin(true_items_location_ids)]
print(high_demand_df.shape)
high_demand_df.head()

(4403, 4)


,date,PULocationID,count,pred
0,2023-01-02,132,6419.0,0
1,2023-01-03,132,6784.0,6430
2,2023-01-04,132,6006.0,5764
3,2023-01-05,132,5617.0,5605
4,2023-01-06,132,4877.0,5618


### evaluate on low demand locations

In [77]:
mean_counts = rides_df.groupby('PULocationID')['count'].mean()
true_items_location_ids = mean_counts[mean_counts < LOW_DEMAND_THRESHOLD].index.tolist()
high_demand_df = result_df[result_df['PULocationID'].isin(true_items_location_ids)]
print(high_demand_df.shape)
high_demand_df.head()

(25347, 4)


,date,PULocationID,count,pred
0,2023-01-02,226,55.0,0
1,2023-01-03,226,78.0,54
2,2023-01-04,226,84.0,70
3,2023-01-05,226,70.0,76
4,2023-01-06,226,83.0,66


In [78]:
high_demand_df = high_demand_df[high_demand_df['pred'] > 0]

evaluate_model(high_demand_df['count'], high_demand_df['pred'])

mape = 471417778966625.25
mae = 4.952376370983259
rmse = 11.805699598311831


### save results

In [75]:
result_df.to_excel(OUTPUT_PATH)